In [2]:
import time
import subprocess

model_ids = [
    # 'google/gemma-2-2b-it',
    # 'mistralai/Mistral-7B-Instruct-v0.3',
    # "mistralai/Mixtral-8x7B-Instruct-v0.1",
    # "meta-llama/Meta-Llama-3.1-8B-Instruct",
    # "qwen/Qwen2-7B-Instruct",
    # "HuggingFaceH4/zephyr-7b-beta",
    # "meta-llama/Llama-2-7b-chat-hf",
    # "ContinuousAT/Llama-2-7B-CAT",
    # "ContinuousAT/Zephyr-CAT",
    # "ContinuousAT/Phi-CAT",
    'microsoft/Phi-3-mini-4k-instruct',
    # 'cais/zephyr_7b_r2d2',
    # 'GraySwanAI/Llama-3-8B-Instruct-RR',
    # 'GraySwanAI/Mistral-7B-Instruct-RR',
    # 'lmsys/vicuna-13b-v1.5'
]

dtypes = ["bfloat16", "float16", "int4", "int8"]
devices = ["auto", "cuda"]
precisions = ["highest", "high", "medium"]

for model_id in model_ids:
    for dtype in dtypes:
        for device in devices:
            for precision in precisions:
                # %run 'eff.py' --model_id=$model_id --dtype=$dtype --device=$device --precision=$precision
                res = subprocess.run(['python', 'eff.py', '--model_id', model_id, '--dtype', dtype, '--device', device, '--precision', precision])
                print(res)
for model_id in model_ids:
    for dtype in dtypes:
        for device in devices:
            for precision in precisions:
                # %run 'eff.py' --model_id=$model_id --dtype=$dtype --compile --device=$device --precision=$precision
                res = subprocess.run(
                    [
                        "python",
                        "eff.py",
                        "--model_id",
                        model_id,
                        "--dtype",
                        dtype,
                        "--compile",
                        "--device",
                        device,
                        "--precision",
                        precision,
                    ]
                )
                print(res)

Namespace(model_id='microsoft/Phi-3-mini-4k-instruct', dtype='bfloat16', compile=False, device='auto', precision='highest')


 30%|███       | 60/198 [00:34<01:18,  1.76it/s]


OOM at 5200 microsoft/Phi-3-mini-4k-instruct bfloat16 False auto highest
CompletedProcess(args=['python', 'eff.py', '--model_id', 'microsoft/Phi-3-mini-4k-instruct', '--dtype', 'bfloat16', '--device', 'auto', '--precision', 'highest'], returncode=0)
Namespace(model_id='microsoft/Phi-3-mini-4k-instruct', dtype='bfloat16', compile=False, device='auto', precision='high')


 30%|███       | 60/198 [00:34<01:18,  1.75it/s]


OOM at 5200 microsoft/Phi-3-mini-4k-instruct bfloat16 False auto high
CompletedProcess(args=['python', 'eff.py', '--model_id', 'microsoft/Phi-3-mini-4k-instruct', '--dtype', 'bfloat16', '--device', 'auto', '--precision', 'high'], returncode=0)


KeyboardInterrupt: 

In [1]:
from transformers import AutoTokenizer
from datasets.dataset import Dataset, AdvBehaviorsConfig
d = Dataset.from_name("adv_behaviors")(
    AdvBehaviorsConfig(
        "adv_behaviors",
        messages_path="/nfs/staff-ssd/beyer/llm-quick-check/data/behavior_datasets/harmbench_behaviors_text_all.csv",
        targets_path="/nfs/staff-ssd/beyer/llm-quick-check/data/optimizer_targets/harmbench_targets_text.json",
        seed=0,
        categories=[
            "chemical_biological",
            "illegal",
            "misinformation_disinformation",
            "harmful",
            "harassment_bullying",
            "cybercrime_intrusion",
        ],
    )
)



tokenizer_ids = [
    'google/gemma-2-2b-it',
    'mistralai/Mistral-7B-Instruct-v0.3',
    'meta-llama/Meta-Llama-3.1-8B-Instruct',
    'qwen/Qwen2-7B-Instruct',
    'HuggingFaceH4/zephyr-7b-beta',
    'meta-llama/Llama-2-7b-chat-hf',
    'microsoft/Phi-3-mini-4k-instruct',
    'cais/zephyr_7b_r2d2',
    'GraySwanAI/Llama-3-8B-Instruct-RR',
    'GraySwanAI/Mistral-7B-Instruct-RR',
    # 'lmsys/vicuna-13b-v1.5'
]
length_dict = {}
sizes = {}
for tokenizer_id in tokenizer_ids:
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)
    sizes[tokenizer_id] = tokenizer.vocab_size
    lengths = []
    for msg, target in d:
        lengths.append(len(tokenizer.apply_chat_template([msg, {'role': 'assistant', 'content': target}])))
    length_dict[tokenizer_id] = lengths
for k, v in length_dict.items():
    print(f"{k:<40} avg: {sum(v) / len(v):04.2f}, max: {max(v):<4d} vocab_size: {sizes[k]:<6d}")


google/gemma-2-2b-it                     avg: 160.79, max: 971  vocab_size: 256000
mistralai/Mistral-7B-Instruct-v0.3       avg: 175.18, max: 1145 vocab_size: 32768 
meta-llama/Meta-Llama-3.1-8B-Instruct    avg: 179.60, max: 1011 vocab_size: 128000
qwen/Qwen2-7B-Instruct                   avg: 169.17, max: 1038 vocab_size: 151643
HuggingFaceH4/zephyr-7b-beta             avg: 190.41, max: 1160 vocab_size: 32000 
meta-llama/Llama-2-7b-chat-hf            avg: 186.96, max: 1161 vocab_size: 32000 
microsoft/Phi-3-mini-4k-instruct         avg: 181.96, max: 1156 vocab_size: 32000 
cais/zephyr_7b_r2d2                      avg: 190.41, max: 1160 vocab_size: 32000 
GraySwanAI/Llama-3-8B-Instruct-RR        avg: 154.60, max: 986  vocab_size: 128000
GraySwanAI/Mistral-7B-Instruct-RR        avg: 180.18, max: 1150 vocab_size: 32000 


In [ ]:
from transformers import AutoModelForCausalLM

In [ ]:
model = AutoModelForCausalLM.from_pretrained('microsoft/Phi-3-mini-4k-instruct')

In [ ]:
model.config